# Feature-based predictor
This notebook is based on [Andrew Lukyanenko](https://www.kaggle.com/artgor)'s [kernel](https://www.kaggle.com/artgor/earthquakes-fe-more-features-and-samples).

## General information
Correctly predicting earthquakes is very important for preventing deaths and damage to infrastructure. In this competition we try to predict time left to the next laboratory earthquake based on seismic signal data. Training data represents one huge signal, but in test data we have many separate chunks, for each of which we need to predict time to failure.

## Preliminaries
Let's import everything we need:

In [1]:
import gc
import os
import time
import random
import datetime
import warnings

import numpy as np
import pandas as pd
import xgboost as xgb
import seaborn as sns
import lightgbm as lgb
import tensorflow as tf
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook
from catboost import CatBoostRegressor
from scipy import stats
from scipy.signal import hilbert, hann, convolve

#from tensorflow import keras
from keras import layers, Sequential, callbacks, backend
from keras.utils import multi_gpu_model
from keras.models import Sequential
from keras.layers import Dense
from keras.regularizers import l1, l2

#from tensorflow.keras import layers


from sklearn.model_selection import ParameterGrid
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor
from sklearn.svm import NuSVR, SVR
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold, GridSearchCV, cross_val_score
from utils import freq_from_crossings, freq_from_fft
from features import gpi

Using TensorFlow backend.


In [2]:
%matplotlib inline
pd.options.display.precision = 15
warnings.filterwarnings('ignore')
random.seed(1013)

## Training data

In [3]:
%%time
train = pd.read_csv('../input/train.csv', dtype={'acoustic_data': np.int16, 'time_to_failure': np.float32})
fs = 4000000

KeyboardInterrupt: 

In [4]:
print(f'Train: rows:{train.shape[0]} cols:{train.shape[1]}')

NameError: name 'train' is not defined

Let's visualize it

In [ ]:
train_acoustic_data_small = train['acoustic_data'].values[::50]
train_time_to_failure_small = train['time_to_failure'].values[::50]

fig, ax1 = plt.subplots(figsize=(16, 8))
plt.title('Trends of acoustic_data and time_to_failure. 2% of the data (sampled)')
plt.plot(train_acoustic_data_small, color='b')
ax1.set_ylabel('acoustic_data', color='b')
plt.legend(['acoustic_data'])
ax2 = ax1.twinx()
plt.plot(train_time_to_failure_small, color='g')
ax2.set_ylabel('time_to_failure', color='g')
plt.legend(['time_to_failure'], loc=(0.875, 0.9))
plt.grid(False)

del train_acoustic_data_small
del train_time_to_failure_small

- We can see that usually acoustic data shows huge fluctuations just before the failure and the nature of data is cyclical;
- Another important point: visually failures can be predicted as cases when huge fluctuations in signal are followes by small signal values. This could be useful for predicting "time_to_failure" changes from 0 to high values;
- I thought that comparing max values of signal in a segment to some threshold value (1000 or 2000) could be useful, but it didn't work;

In [ ]:
plt.figure(figsize=(16, 8))
plt.hist(train['time_to_failure'].values[::50], bins='auto', density=True)  # arguments are passed to np.histogram
plt.title('Histogram of time_to_failure')
plt.xlabel('time_to_failure')
plt.ylabel('count')
plt.show()

- The time to training is not uniforally distribured on the training set. Resampling could be interesting.

## Feature generation
- Usual aggregations: mean, std, min and max;
- Average difference between the consequitive values in absolute and percent values;
- Absolute min and max vallues;
- Aforementioned aggregations for first and last 10000 and 50000 values - I think these data should be useful;
- Max value to min value and their differencem also count of values bigger that 500 (arbitrary threshold);
- Quantile features from this kernel: https://www.kaggle.com/andrekos/basic-feature-benchmark-with-quantiles
- Trend features from this kernel: https://www.kaggle.com/jsaguiar/baseline-with-abs-and-trend-features
- Rolling features from this kernel: https://www.kaggle.com/wimwim/rolling-quantiles

In [ ]:
def add_trend_feature(arr, abs_values=False):
    idx = np.array(range(len(arr)))
    if abs_values:
        arr = np.abs(arr)
    lr = LinearRegression()
    lr.fit(idx.reshape(-1, 1), arr)
    return lr.coef_[0]

In [ ]:
def sta_lta_ratio(x, length_sta, length_lta, method='original'):
    if method=='original':
        sta = np.cumsum(x ** 2)
        # Convert to float
        sta = np.require(sta, dtype=np.float)
        # Copy for LTA
        lta = sta.copy()
        # Compute the STA and the LTA
        sta[length_sta:] = sta[length_sta:] - sta[:-length_sta]
        sta /= length_sta
        lta[length_lta:] = lta[length_lta:] - lta[:-length_lta]
        lta /= length_lta
        # Pad zeros
        sta[:length_lta - 1] = 0
        # Avoid division by zero by setting zero values to tiny float
        dtiny = np.finfo(0.0).tiny
        idx = lta < dtiny
        lta[idx] = dtiny
        ratio = sta / lta
        
    elif method == 'modified':
        x_abs = np.abs(x)
        # Convert to float
        x_abs = np.require(x_abs, dtype=np.float)
        # Compute the STA and the LTA
        sta = np.cumsum(x_abs)
        sta[length_sta:] = sta[length_sta:] - sta[:-length_sta]
        sta = sta[length_sta - 1:] / length_sta
        sta = sta[:-(length_lta-length_sta)]
        lta = x_abs.copy()
        lta = np.cumsum(lta)
        lta[length_lta:] = lta[length_lta:] - lta[:-length_lta]
        lta = lta[length_lta - 1:] / length_lta
        ratio = sta / lta

    return ratio

In [ ]:
def change_rate(x, method='original'):
    if method == 'original':
        rate = np.mean(np.nonzero((np.diff(x) / x[:-1]))[0])
    if method == 'modified':
        change = (np.diff(x) / x[:-1]).values
        change = change[np.nonzero(change)[0]]
        change = change[~np.isnan(change)]
        change = change[change != -np.inf]
        change = change[change != np.inf]
        rate = np.mean(change)
    return rate

In [ ]:
def create_features(seg_id, seg, X):
    xc = pd.Series(seg['acoustic_data'].values)
    zc = np.fft.fft(xc)
    
    # Generic stats
    X.loc[seg_id, 'mean'] = xc.mean()
    X.loc[seg_id, 'std'] = xc.std()
    X.loc[seg_id, 'max'] = xc.max()
    X.loc[seg_id, 'min'] = xc.min()
    
    #FFT transform values
    realFFT = np.real(zc)
    imagFFT = np.imag(zc)
    X.loc[seg_id, 'Rmean'] = realFFT.mean()
    X.loc[seg_id, 'Rstd'] = realFFT.std()
    X.loc[seg_id, 'Rmax'] = realFFT.max()
    X.loc[seg_id, 'Rmin'] = realFFT.min()
    X.loc[seg_id, 'Imean'] = imagFFT.mean()
    X.loc[seg_id, 'Istd'] = imagFFT.std()
    X.loc[seg_id, 'Imax'] = imagFFT.max()
    X.loc[seg_id, 'Imin'] = imagFFT.min()
    X.loc[seg_id, 'Rmean_last_5000'] = realFFT[-5000:].mean()
    X.loc[seg_id, 'Rstd__last_5000'] = realFFT[-5000:].std()
    X.loc[seg_id, 'Rmax_last_5000'] = realFFT[-5000:].max()
    X.loc[seg_id, 'Rmin_last_5000'] = realFFT[-5000:].min()
    X.loc[seg_id, 'Rmean_last_15000'] = realFFT[-15000:].mean()
    X.loc[seg_id, 'Rstd_last_15000'] = realFFT[-15000:].std()
    X.loc[seg_id, 'Rmax_last_15000'] = realFFT[-15000:].max()
    X.loc[seg_id, 'Rmin_last_15000'] = realFFT[-15000:].min()
    
    X.loc[seg_id, 'mean_diff'] = np.mean(np.diff(xc))
    X.loc[seg_id, 'mean_abs_diff'] = np.mean(np.abs(np.diff(xc)))
    X.loc[seg_id, 'mean_change_rate'] = change_rate(xc, method='original')
    X.loc[seg_id, 'mean_change_rate_v2'] = change_rate(xc, method='modified')
    X.loc[seg_id, 'abs_max'] = np.abs(xc).max()
    X.loc[seg_id, 'abs_min'] = np.abs(xc).min()
    
    # Classical stats by segment
    X.loc[seg_id, 'std_first_50000'] = xc[:50000].std()
    X.loc[seg_id, 'std_last_50000'] = xc[-50000:].std()
    X.loc[seg_id, 'std_first_10000'] = xc[:10000].std()
    X.loc[seg_id, 'std_last_10000'] = xc[-10000:].std()
    
    X.loc[seg_id, 'avg_first_50000'] = xc[:50000].mean()
    X.loc[seg_id, 'avg_last_50000'] = xc[-50000:].mean()
    X.loc[seg_id, 'avg_first_10000'] = xc[:10000].mean()
    X.loc[seg_id, 'avg_last_10000'] = xc[-10000:].mean()
    
    X.loc[seg_id, 'min_first_50000'] = xc[:50000].min()
    X.loc[seg_id, 'min_last_50000'] = xc[-50000:].min()
    X.loc[seg_id, 'min_first_10000'] = xc[:10000].min()
    X.loc[seg_id, 'min_last_10000'] = xc[-10000:].min()
    
    X.loc[seg_id, 'max_first_50000'] = xc[:50000].max()
    X.loc[seg_id, 'max_last_50000'] = xc[-50000:].max()
    X.loc[seg_id, 'max_first_10000'] = xc[:10000].max()
    X.loc[seg_id, 'max_last_10000'] = xc[-10000:].max()
    
    X.loc[seg_id, 'max_to_min'] = xc.max() / np.abs(xc.min())
    X.loc[seg_id, 'max_to_min_diff'] = xc.max() - np.abs(xc.min())
    X.loc[seg_id, 'count_big'] = len(xc[np.abs(xc) > 500])
    X.loc[seg_id, 'sum'] = xc.sum()
    
    X.loc[seg_id, 'mean_change_rate_first_50000'] = change_rate(xc[:50000], method='original')
    X.loc[seg_id, 'mean_change_rate_last_50000'] = change_rate(xc[-50000:], method='original')
    X.loc[seg_id, 'mean_change_rate_first_10000'] = change_rate(xc[:10000], method='original')
    X.loc[seg_id, 'mean_change_rate_last_10000'] = change_rate(xc[-10000:], method='original')

    X.loc[seg_id, 'mean_change_rate_first_50000_v2'] = change_rate(xc[:50000], method='modified')
    X.loc[seg_id, 'mean_change_rate_last_50000_v2'] = change_rate(xc[-50000:], method='modified')
    X.loc[seg_id, 'mean_change_rate_first_10000_v2'] = change_rate(xc[:10000], method='modified')
    X.loc[seg_id, 'mean_change_rate_last_10000_v2'] = change_rate(xc[-10000:], method='modified')

    X.loc[seg_id, 'q95'] = np.quantile(xc, 0.95)
    X.loc[seg_id, 'q99'] = np.quantile(xc, 0.99)
    X.loc[seg_id, 'q05'] = np.quantile(xc, 0.05)
    X.loc[seg_id, 'q01'] = np.quantile(xc, 0.01)
    
    X.loc[seg_id, 'abs_q95'] = np.quantile(np.abs(xc), 0.95)
    X.loc[seg_id, 'abs_q99'] = np.quantile(np.abs(xc), 0.99)
    X.loc[seg_id, 'abs_q05'] = np.quantile(np.abs(xc), 0.05)
    X.loc[seg_id, 'abs_q01'] = np.quantile(np.abs(xc), 0.01)
    
    X.loc[seg_id, 'trend'] = add_trend_feature(xc)
    X.loc[seg_id, 'abs_trend'] = add_trend_feature(xc, abs_values=True)
    X.loc[seg_id, 'abs_mean'] = np.abs(xc).mean()
    X.loc[seg_id, 'abs_std'] = np.abs(xc).std()
    
    X.loc[seg_id, 'mad'] = xc.mad()
    X.loc[seg_id, 'kurt'] = xc.kurtosis()
    X.loc[seg_id, 'skew'] = xc.skew()
    X.loc[seg_id, 'med'] = xc.median()
    
    X.loc[seg_id, 'Hilbert_mean'] = np.abs(hilbert(xc)).mean()
    X.loc[seg_id, 'Hann_window_mean'] = (convolve(xc, hann(150), mode='same') / sum(hann(150))).mean()    
    
    sta_lta_method = 'original'
    classic_sta_lta1 = sta_lta_ratio(xc, 500, 10000, method=sta_lta_method)
    classic_sta_lta2 = sta_lta_ratio(xc, 5000, 100000, method=sta_lta_method)
    classic_sta_lta3 = sta_lta_ratio(xc, 3333, 6666, method=sta_lta_method)
    classic_sta_lta4 = sta_lta_ratio(xc, 10000, 25000, method=sta_lta_method)
    classic_sta_lta5 = sta_lta_ratio(xc, 50, 1000, method=sta_lta_method)
    classic_sta_lta6 = sta_lta_ratio(xc, 100, 5000, method=sta_lta_method)
    classic_sta_lta7 = sta_lta_ratio(xc, 333, 666, method=sta_lta_method)
    classic_sta_lta8 = sta_lta_ratio(xc, 4000, 10000, method=sta_lta_method)
    
    X.loc[seg_id, 'classic_sta_lta1_mean'] = classic_sta_lta1.mean()
    X.loc[seg_id, 'classic_sta_lta2_mean'] = classic_sta_lta2.mean()
    X.loc[seg_id, 'classic_sta_lta3_mean'] = classic_sta_lta3.mean()
    X.loc[seg_id, 'classic_sta_lta4_mean'] = classic_sta_lta4.mean()
    X.loc[seg_id, 'classic_sta_lta5_mean'] = classic_sta_lta5.mean()
    X.loc[seg_id, 'classic_sta_lta6_mean'] = classic_sta_lta6.mean()
    X.loc[seg_id, 'classic_sta_lta7_mean'] = classic_sta_lta7.mean()
    X.loc[seg_id, 'classic_sta_lta8_mean'] = classic_sta_lta8.mean()

    X.loc[seg_id, 'classic_sta_lta1_q95'] = np.quantile(classic_sta_lta1, 0.95)
    X.loc[seg_id, 'classic_sta_lta2_q95'] = np.quantile(classic_sta_lta2, 0.95)
    X.loc[seg_id, 'classic_sta_lta3_q95'] = np.quantile(classic_sta_lta3, 0.95)
    X.loc[seg_id, 'classic_sta_lta4_q95'] = np.quantile(classic_sta_lta4, 0.95)
    X.loc[seg_id, 'classic_sta_lta5_q95'] = np.quantile(classic_sta_lta5, 0.95)
    X.loc[seg_id, 'classic_sta_lta6_q95'] = np.quantile(classic_sta_lta6, 0.95)
    X.loc[seg_id, 'classic_sta_lta7_q95'] = np.quantile(classic_sta_lta7, 0.95)
    X.loc[seg_id, 'classic_sta_lta8_q95'] = np.quantile(classic_sta_lta8, 0.95)   

    X.loc[seg_id, 'classic_sta_lta1_q05'] = np.quantile(classic_sta_lta1, 0.05)
    X.loc[seg_id, 'classic_sta_lta2_q05'] = np.quantile(classic_sta_lta2, 0.05)
    X.loc[seg_id, 'classic_sta_lta3_q05'] = np.quantile(classic_sta_lta3, 0.05)
    X.loc[seg_id, 'classic_sta_lta4_q05'] = np.quantile(classic_sta_lta4, 0.05)
    X.loc[seg_id, 'classic_sta_lta5_q05'] = np.quantile(classic_sta_lta5, 0.05)
    X.loc[seg_id, 'classic_sta_lta6_q05'] = np.quantile(classic_sta_lta6, 0.05)
    X.loc[seg_id, 'classic_sta_lta7_q05'] = np.quantile(classic_sta_lta7, 0.05)
    X.loc[seg_id, 'classic_sta_lta8_q05'] = np.quantile(classic_sta_lta8, 0.05)

    sta_lta_method = 'modified'
    classic_sta_lta1 = sta_lta_ratio(xc, 500, 10000, method=sta_lta_method)
    classic_sta_lta2 = sta_lta_ratio(xc, 5000, 100000, method=sta_lta_method)
    classic_sta_lta3 = sta_lta_ratio(xc, 3333, 6666, method=sta_lta_method)
    classic_sta_lta4 = sta_lta_ratio(xc, 10000, 25000, method=sta_lta_method)
    classic_sta_lta5 = sta_lta_ratio(xc, 50, 1000, method=sta_lta_method)
    classic_sta_lta6 = sta_lta_ratio(xc, 100, 5000, method=sta_lta_method)
    classic_sta_lta7 = sta_lta_ratio(xc, 333, 666, method=sta_lta_method)
    classic_sta_lta8 = sta_lta_ratio(xc, 4000, 10000, method=sta_lta_method)
    
    X.loc[seg_id, 'modified_sta_lta1_mean'] = classic_sta_lta1.mean()
    X.loc[seg_id, 'modified_sta_lta2_mean'] = classic_sta_lta2.mean()
    X.loc[seg_id, 'modified_sta_lta3_mean'] = classic_sta_lta3.mean()
    X.loc[seg_id, 'modified_sta_lta4_mean'] = classic_sta_lta4.mean()
    X.loc[seg_id, 'modified_sta_lta5_mean'] = classic_sta_lta5.mean()
    X.loc[seg_id, 'modified_sta_lta6_mean'] = classic_sta_lta6.mean()
    X.loc[seg_id, 'modified_sta_lta7_mean'] = classic_sta_lta7.mean()
    X.loc[seg_id, 'modified_sta_lta8_mean'] = classic_sta_lta8.mean()

    X.loc[seg_id, 'modified_sta_lta1_q95'] = np.quantile(classic_sta_lta1, 0.95)
    X.loc[seg_id, 'modified_sta_lta2_q95'] = np.quantile(classic_sta_lta2, 0.95)
    X.loc[seg_id, 'modified_sta_lta3_q95'] = np.quantile(classic_sta_lta3, 0.95)
    X.loc[seg_id, 'modified_sta_lta4_q95'] = np.quantile(classic_sta_lta4, 0.95)
    X.loc[seg_id, 'modified_sta_lta5_q95'] = np.quantile(classic_sta_lta5, 0.95)
    X.loc[seg_id, 'modified_sta_lta6_q95'] = np.quantile(classic_sta_lta6, 0.95)
    X.loc[seg_id, 'modified_sta_lta7_q95'] = np.quantile(classic_sta_lta7, 0.95)
    X.loc[seg_id, 'modified_sta_lta8_q95'] = np.quantile(classic_sta_lta8, 0.95)   

    X.loc[seg_id, 'modified_sta_lta1_q05'] = np.quantile(classic_sta_lta1, 0.05)
    X.loc[seg_id, 'modified_sta_lta2_q05'] = np.quantile(classic_sta_lta2, 0.05)
    X.loc[seg_id, 'modified_sta_lta3_q05'] = np.quantile(classic_sta_lta3, 0.05)
    X.loc[seg_id, 'modified_sta_lta4_q05'] = np.quantile(classic_sta_lta4, 0.05)
    X.loc[seg_id, 'modified_sta_lta5_q05'] = np.quantile(classic_sta_lta5, 0.05)
    X.loc[seg_id, 'modified_sta_lta6_q05'] = np.quantile(classic_sta_lta6, 0.05)
    X.loc[seg_id, 'modified_sta_lta7_q05'] = np.quantile(classic_sta_lta7, 0.05)
    X.loc[seg_id, 'modified_sta_lta8_q05'] = np.quantile(classic_sta_lta8, 0.05)

    X.loc[seg_id, 'Moving_average_700_mean'] = xc.rolling(window=700).mean().mean(skipna=True)
    X.loc[seg_id, 'Moving_average_1500_mean'] = xc.rolling(window=1500).mean().mean(skipna=True)
    X.loc[seg_id, 'Moving_average_3000_mean'] = xc.rolling(window=3000).mean().mean(skipna=True)
    X.loc[seg_id, 'Moving_average_6000_mean'] = xc.rolling(window=6000).mean().mean(skipna=True)
    X.loc[seg_id, 'Moving_average_30000_mean'] = xc.rolling(window=30000).mean().mean(skipna=True)

    ewma = pd.Series.ewm
    X.loc[seg_id, 'exp_Moving_average_300_mean'] = ewma(xc, span=300).mean().mean(skipna=True)
    X.loc[seg_id, 'exp_Moving_average_3000_mean'] = ewma(xc, span=3000).mean().mean(skipna=True)
    X.loc[seg_id, 'exp_Moving_average_6000_mean'] = ewma(xc, span=6000).mean().mean(skipna=True)
    X.loc[seg_id, 'exp_Moving_average_30000_mean'] = ewma(xc, span=30000).mean().mean(skipna=True)

    # rdg: TODO it seems a parameter to tune
    no_of_std = 2
    X.loc[seg_id, 'MA_700MA_std_mean'] = xc.rolling(window=700).std().mean()
    X.loc[seg_id, 'MA_700MA_BB_high_mean'] = (X.loc[seg_id, 'Moving_average_700_mean']
                                              + no_of_std * X.loc[seg_id, 'MA_700MA_std_mean']).mean()
    X.loc[seg_id, 'MA_700MA_BB_low_mean'] = (X.loc[seg_id, 'Moving_average_700_mean']
                                             - no_of_std * X.loc[seg_id, 'MA_700MA_std_mean']).mean()
    X.loc[seg_id, 'MA_400MA_std_mean'] = xc.rolling(window=400).std().mean()
    X.loc[seg_id, 'MA_400MA_BB_high_mean'] = (X.loc[seg_id, 'Moving_average_700_mean']
                                              + no_of_std * X.loc[seg_id, 'MA_400MA_std_mean']).mean()
    X.loc[seg_id, 'MA_400MA_BB_low_mean'] = (X.loc[seg_id, 'Moving_average_700_mean']
                                             - no_of_std * X.loc[seg_id, 'MA_400MA_std_mean']).mean()
    X.loc[seg_id, 'MA_1000MA_std_mean'] = xc.rolling(window=1000).std().mean()
    
    X.loc[seg_id, 'iqr'] = np.subtract(*np.percentile(xc, [75, 25]))
    X.loc[seg_id, 'q999'] = np.quantile(xc, 0.999)
    X.loc[seg_id, 'q001'] = np.quantile(xc, 0.001)
    X.loc[seg_id, 'ave10'] = stats.trim_mean(xc, 0.1)

    X.loc[seg_id, 'freq_cross_first_50000'] = freq_from_crossings(xc.values[:50000], fs)
    X.loc[seg_id, 'freq_cross_last_50000'] = freq_from_crossings(xc.values[-50000:], fs)
    X.loc[seg_id, 'freq_cross_first_10000'] = freq_from_crossings(xc.values[:10000], fs)
    X.loc[seg_id, 'freq_cross_last_10000'] = freq_from_crossings(xc.values[-10000:], fs)
    
    for windows in [10, 100, 1000]:
        x_roll_std = xc.rolling(windows).std().dropna().values
        x_roll_mean = xc.rolling(windows).mean().dropna().values
        
        X.loc[seg_id, 'ave_roll_std_' + str(windows)] = x_roll_std.mean()
        X.loc[seg_id, 'std_roll_std_' + str(windows)] = x_roll_std.std()
        X.loc[seg_id, 'max_roll_std_' + str(windows)] = x_roll_std.max()
        X.loc[seg_id, 'min_roll_std_' + str(windows)] = x_roll_std.min()
        X.loc[seg_id, 'q01_roll_std_' + str(windows)] = np.quantile(x_roll_std, 0.01)
        X.loc[seg_id, 'q05_roll_std_' + str(windows)] = np.quantile(x_roll_std, 0.05)
        X.loc[seg_id, 'q95_roll_std_' + str(windows)] = np.quantile(x_roll_std, 0.95)
        X.loc[seg_id, 'q99_roll_std_' + str(windows)] = np.quantile(x_roll_std, 0.99)
        X.loc[seg_id, 'av_change_abs_roll_std_' + str(windows)] = np.mean(np.abs(np.diff(x_roll_std)))
        X.loc[seg_id, 'av_change_rate_roll_std_' + str(windows)] = change_rate(pd.Series(x_roll_std), 
                                                                               method='original')
        X.loc[seg_id, 'av_change_rate_roll_std_' + str(windows) + 'v2'] = change_rate(pd.Series(x_roll_std),
                                                                                      method='modified')
        X.loc[seg_id, 'abs_max_roll_std_' + str(windows)] = np.abs(x_roll_std).max()
        X.loc[seg_id, 'ave_roll_mean_' + str(windows)] = x_roll_mean.mean()
        X.loc[seg_id, 'std_roll_mean_' + str(windows)] = x_roll_mean.std()
        X.loc[seg_id, 'max_roll_mean_' + str(windows)] = x_roll_mean.max()
        X.loc[seg_id, 'min_roll_mean_' + str(windows)] = x_roll_mean.min()
        X.loc[seg_id, 'q01_roll_mean_' + str(windows)] = np.quantile(x_roll_mean, 0.01)
        X.loc[seg_id, 'q05_roll_mean_' + str(windows)] = np.quantile(x_roll_mean, 0.05)
        X.loc[seg_id, 'q95_roll_mean_' + str(windows)] = np.quantile(x_roll_mean, 0.95)
        X.loc[seg_id, 'q99_roll_mean_' + str(windows)] = np.quantile(x_roll_mean, 0.99)
        X.loc[seg_id, 'av_change_abs_roll_mean_' + str(windows)] = np.mean(np.abs(np.diff(x_roll_mean)))
        X.loc[seg_id, 'av_change_rate_roll_mean_' + str(windows)] = change_rate(pd.Series(x_roll_mean),
                                                                               method = 'original')
        X.loc[seg_id, 'av_change_rate_roll_mean_' + str(windows) + '_v2'] = change_rate(pd.Series(x_roll_mean),
                                                                                       method = 'modified')
        X.loc[seg_id, 'abs_max_roll_mean_' + str(windows)] = np.abs(x_roll_mean).max()

## Sampling the training set
Since the testign set is build of sequences of 150.000 samples, let's chop the training set on comparable chunks. 

In [ ]:
# Create a training file with simple derived features
segment_size = 150000

In [ ]:
def generate_segment_start_ids(sampling_method):
    if sampling_method == 'uniform':
        # With this approach we obtain 4194 segments
        num_segments_training = int(np.floor(train.shape[0] / segment_size))
        segment_start_ids = [i * segment_size for i in range(num_segments_training)]
    elif sampling_method == 'uniform_no_jump':
        # With this approach we obtain 4178 segments (99.5% of 'uniform')
        already_sampled = np.full(train.shape[0], False)
        num_segments_training = int(np.floor(train.shape[0] / segment_size))
        time_to_failure_jumps = np.diff(train['time_to_failure'].values)
        num_good_segments_found = 0
        segment_start_ids = []
        for i in range(num_segments_training):
            idx = i * segment_size
            # Detect if there is a discontinuity on the time_to_failure signal within the segment
            max_jump = np.max(time_to_failure_jumps[idx:idx + segment_size])
            if max_jump < 5:
                segment_start_ids.append(idx)
                num_good_segments_found += 1
            else:
                print(f'Rejected candidate segment since max_jump={max_jump}')
        segment_start_ids.sort()
    elif sampling_method == 'random_no_jump':
        # With this approach we obtain 4194 segments
        num_segments_training = int(np.floor(train.shape[0] / segment_size)) #arbitrary choice
        time_to_failure_jumps = np.diff(train['time_to_failure'].values)
        num_good_segments_found = 0
        segment_start_ids = []
        while num_segments_training != num_good_segments_found:
            # Generate a random sampling position
            idx = random.randint(0, train.shape[0] - segment_size - 1)
            # Detect if there is a discontinuity on the time_to_failure signal within the segment
            max_jump = np.max(time_to_failure_jumps[idx:idx + segment_size])
            if max_jump < 5:
                segment_start_ids.append(idx)
                num_good_segments_found += 1
            else:
                print(f'Rejected candidate segment since max_jump={max_jump}')
        segment_start_ids.sort()
    else:
        raise NameError('Method does not exist')
    return segment_start_ids

Currently, we have two strategies implemented.
- Uniform sampling: Here the training set is built from consecutive chunks of data. No special care is taken into what's the content of each segment.
- Uniform sampling with rejection: Here the training set is built from consecutive chunks of data. There is a control to avoid having in a segment the jump of the `time_to_failure` signal from zero to a high value.
- Random sampling with rejection: Here the training set is built from sampling randomly the data. There is a control to avoid having in a segment the jump of the `time_to_failure` signal from zero to a high value. The segments will most likely overlap.

Let's visualize what happens to the distribution of the `time_to_failure` on teh different samplign strategies:

In [ ]:
print(f'Generating uniformly sampled training set')
segment_start_ids_uniform = generate_segment_start_ids('uniform')

print(f'Generating uniformly sampled training set excluding discontinuities in time_to_failure.')
segment_start_ids_uniform_no_jump = generate_segment_start_ids('uniform_no_jump')

print(f'Generating randomly sampled training set excluding discontinuities in time_to_failure.')
print(f'This method may yield overlaping segments')
segment_start_ids_random_no_jump = generate_segment_start_ids('random_no_jump')


y_tr_samples_uniform = train['time_to_failure'].values[np.array(
    segment_start_ids_uniform) + segment_size - 1]
y_tr_samples_uniform_no_jump = train['time_to_failure'].values[
    np.array(segment_start_ids_uniform_no_jump) + segment_size - 1]
y_tr_samples_random_no_jump = train['time_to_failure'].values[
    np.array(segment_start_ids_random_no_jump) + segment_size - 1]

plt.subplots(figsize=(16, 16))
plt.subplot(3, 3, 1)
plt.plot(segment_start_ids_uniform)
plt.title('segment_start_ids_uniform')
plt.subplot(3, 3, 2)
plt.plot(segment_start_ids_uniform_no_jump)
plt.title('segment_start_ids_uniform_no_jump')
plt.subplot(3, 3, 3)
plt.plot(segment_start_ids_random_no_jump)
plt.title('segment_start_ids_random_no_jump')

plt.subplot(3, 3, 4)
plt.plot(y_tr_samples_uniform)
plt.title('time_to_failure values (all)')
plt.subplot(3, 3, 5)
plt.plot(y_tr_samples_uniform_no_jump)
plt.title('time_to_failure values (uniform sampling no jump)')
plt.subplot(3, 3, 6)
plt.plot(y_tr_samples_random_no_jump)
plt.title('time_to_failure values (random sampling no jump)')

plt.subplot(3, 3, 7)
plt.hist(y_tr_samples_uniform, bins='auto', alpha=0.5, density=True)
plt.hist(train['time_to_failure'].values[::50], bins='auto', alpha=0.5, density=True)
plt.title('With discontinuities (contiguous)')
plt.legend(['Sampled', 'All data'])

plt.subplot(3, 3, 8)
plt.hist(y_tr_samples_uniform_no_jump, bins='auto', alpha=0.5, density=True)
plt.hist(train['time_to_failure'].values[::50], bins='auto', alpha=0.5, density=True)
plt.title('Discarding discontinuities (contiguous)')
plt.legend(['Sampled', 'All data'])

plt.subplot(3, 3, 9)
plt.hist(y_tr_samples_random_no_jump, bins='auto', alpha=0.5, density=True)
plt.title('Discarding discontinuities (rand)')
plt.hist(train['time_to_failure'].values[::50], bins='auto', alpha=0.5, density=True)
plt.legend(['Sampled', 'All data'])
plt.show()

del segment_start_ids_uniform
del segment_start_ids_uniform_no_jump
del segment_start_ids_random_no_jump
del y_tr_samples_uniform
del y_tr_samples_uniform_no_jump
del y_tr_samples_random_no_jump

Finally, we choose one sampling method (`uniform`) and run with it

In [ ]:
segment_start_ids = generate_segment_start_ids('uniform_no_jump')

In [ ]:
X_tr = pd.DataFrame(index=range(len(segment_start_ids)), dtype=np.float64)
y_tr = pd.DataFrame(index=range(len(segment_start_ids)), dtype=np.float64, columns=['time_to_failure'])

Compute all features for all segments (WARNING: it takes time)

In [ ]:
for idx in tqdm_notebook(range(len(segment_start_ids))):        
    seg_id = segment_start_ids[idx]
    seg = train.iloc[seg_id:seg_id + segment_size]
    create_features(idx, seg, X_tr)
    y_tr.loc[idx, 'time_to_failure'] = seg['time_to_failure'].values[-1]

In [ ]:
print(f'{X_tr.shape[0]} samples in new train data and {X_tr.shape[1]} columns.')

Let's take a look of how some of these features look like

In [ ]:
plt.subplots(figsize=(44, 24))
cols = list(np.abs(X_tr.corrwith(y_tr['time_to_failure'])).sort_values(ascending=False).head(24).index)
for i, col in enumerate(cols):
    ax1 = plt.subplot(6, 4, i + 1)
    plt.plot(X_tr[col], color='blue')
    plt.title(col)
    ax1.set_ylabel(col, color='b')

    ax2 = ax1.twinx()
    plt.plot(y_tr, color='g')
    ax2.set_ylabel('time_to_failure', color='g')
    plt.legend([col, 'time_to_failure'])
    plt.grid(False)

Sometimes, there are features that exploded, which show as `np.inf` in the array. Let's just replace them by the mean values. TODO: check if this is the appropiate strategy. 

In [ ]:
# Sanity check
means_dict = {}
for col in X_tr.columns:
    if X_tr[col].isnull().any():
        print(col)
        mean_value = X_tr.loc[X_tr[col] != -np.inf, col].mean()
        X_tr.loc[X_tr[col] == -np.inf, col] = mean_value
        X_tr[col] = X_tr[col].fillna(mean_value)
        means_dict[col] = mean_value

## Reading test data

In [ ]:
submission = pd.read_csv('../input/sample_submission.csv', index_col='seg_id')
X_test = pd.DataFrame(columns=X_tr.columns, dtype=np.float64, index=submission.index)

In [ ]:
plt.figure(figsize=(22, 16))
for i, seg_id in enumerate(tqdm_notebook(X_test.index)):
    seg = pd.read_csv('../input/test/' + seg_id + '.csv')
    create_features(seg_id, seg, X_test)

In [ ]:
# Sanity check
for col in X_test.columns:
    if X_test[col].isnull().any():
        X_test.loc[X_test[col] == -np.inf, col] = means_dict[col]
        X_test[col] = X_test[col].fillna(means_dict[col])

## Scale data

In [ ]:
#scaler = StandardScaler()
#scaler.fit(X_tr)
#X_train_scaled = pd.DataFrame(scaler.transform(X_tr), columns=X_tr.columns)
#X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

In [ ]:
#Only real difference is scaling the lot in one go rather than scling train and then using this to scale test
alldata = pd.concat([X_tr, X_test])
scaler = StandardScaler()
alldata = pd.DataFrame(scaler.fit_transform(alldata), columns=alldata.columns)
X_train_scaled = alldata[:X_tr.shape[0]]
X_test_scaled = alldata[X_tr.shape[0]:]

## Building models

In [ ]:
def grid_search_cv(estimator, grid, features, target, num_folds):
    """Return the best hyperparameters combination in grid."""
    t0 = time.time()
    reg = GridSearchCV(estimator, grid, cv=num_folds, scoring='neg_mean_absolute_error', n_jobs=-1, verbose=10)
    reg.fit(features, target)
    
    t0 = time.time() - t0
    print("Best CV score: {:.4f}, time: {:.1f}s".format(-reg.best_score_, t0))
    print(reg.best_params_)
    return reg.best_params_, reg.cv_results_

In [ ]:
def train_model(X, X_test, y, folds, params=None, model_type='lgb',
                model=None, show_scatter=False):

    oof = np.zeros(len(X))
    prediction = np.zeros(len(X_test))
    scores = []
    n_fold = folds.get_n_splits()
    
    feature_importance = pd.DataFrame()
    for fold_n, (train_index, valid_index) in enumerate(folds.split(X)):
        print('Fold', fold_n, 'started at', time.ctime())
        X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
        y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]

        if model_type == 'nn':
            model = Sequential()
            model.add(layers.Dense(1024, input_dim=216, activation=tf.nn.relu))
            model.add(layers.Dropout(0.8))
            model.add(layers.Dense(256, activation=tf.nn.relu))
            model.add(layers.Dropout(0.8))
            model.add(layers.Dense(256, activation=tf.nn.relu))
            model.add(layers.Dropout(0.8))
            model.add(layers.Dense(16, activation=tf.nn.relu))
            model.add(layers.Dense(1))

            model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
            EPOCHS = 1000
            early_stop = callbacks.EarlyStopping(monitor='mean_absolute_error', patience=100)

            history = model.fit(
                X_train,
                y_train,
                epochs=EPOCHS,
                validation_data = (X_valid, y_valid), 
                verbose=0,
                callbacks=[early_stop, PrintDot()])
            hist = pd.DataFrame(history.history)
            val_score = hist['val_mean_absolute_error'].iloc[-1]
            print(f'val_score={val_score}')
            plot_history(history)
        
            y_pred_valid = model.predict(X_valid).reshape(-1,)
            y_pred = model.predict(X_test).reshape(-1,)

            score = mean_absolute_error(y_valid, y_pred_valid)
            print(f'Fold {fold_n}. MAE: {score:.4f}.')
        
        if model_type == 'lgb':
            model = lgb.LGBMRegressor(**params, n_estimators = 50000, n_jobs = 32)
            model.fit(X_train, y_train, 
                      eval_set=[(X_train, y_train), (X_valid, y_valid)],
                      eval_metric='mae',
                      verbose=10000,
                      early_stopping_rounds=2000)
            
            y_pred_valid = model.predict(X_valid)
            y_pred = model.predict(X_test, num_iteration=model.best_iteration_)
            
        if model_type == 'xgb':
            train_data = xgb.DMatrix(data=X_train, label=y_train, feature_names=X.columns)
            valid_data = xgb.DMatrix(data=X_valid, label=y_valid, feature_names=X.columns)

            watchlist = [(train_data, 'train'), (valid_data, 'valid_data')]
            model = xgb.train(dtrain=train_data,
                              num_boost_round=20000,
                              evals=watchlist,
                              early_stopping_rounds=200,
                              verbose_eval=500,
                              params=params)

            y_pred_valid = model.predict(xgb.DMatrix(X_valid, feature_names=X.columns),
                                         ntree_limit=model.best_ntree_limit)
            y_pred = model.predict(xgb.DMatrix(X_test, feature_names=X.columns),
                                   ntree_limit=model.best_ntree_limit)
        
        if model_type == 'sklearn':
            model = model
            model.fit(X_train, y_train)
            
            y_pred_valid = model.predict(X_valid).reshape(-1,)
            score = mean_absolute_error(y_valid, y_pred_valid)
            y_pred = model.predict(X_test).reshape(-1,)
            print(f'Fold {fold_n}. MAE: {score:.4f}.')
            print('')
        
        if model_type == 'cat':
            model = CatBoostRegressor(iterations=20000,  eval_metric='MAE', task_type='GPU', **params)
            model.fit(X_train, y_train, eval_set=(X_valid, y_valid), cat_features=[], use_best_model=True,
                      verbose=False)

            y_pred_valid = model.predict(X_valid)
            y_pred = model.predict(X_test)
        
        if model_type == 'gdi':
            y_pred_valid = gpi(X_valid).values
            y_pred = gpi(X_test).values
        
        oof[valid_index] = y_pred_valid.reshape(-1,)
        scores.append(mean_absolute_error(y_valid, y_pred_valid))
        
        prediction += y_pred    
        
        if model_type == 'lgb':
            # feature importance
            fold_importance = pd.DataFrame()
            fold_importance['feature'] = X.columns
            fold_importance['importance'] = model.feature_importances_
            fold_importance['fold'] = fold_n + 1
            feature_importance = pd.concat([feature_importance, fold_importance], axis=0)

    prediction /= n_fold
    
    if show_scatter:
        fig, axis = plt.subplots(1, 2, figsize=(12,5))
        ax1, ax2 = axis
        ax1.set_xlabel('actual')
        ax1.set_ylabel('predicted')
        ax2.set_xlabel('train index')
        ax2.set_ylabel('time to failure')
        
        ax1.scatter(y, oof, color='brown')
        ax1.plot([(0, 0), (20, 20)], [(0, 0), (20, 20)], color='blue')

        ax2.plot(y, color='blue', label='y_train')
        ax2.plot(oof, color='orange')
    
    print('CV mean score: {0:.4f}, std: {1:.4f}.'.format(np.mean(scores), np.std(scores)))
    
    if model_type == 'lgb':
        feature_importance['importance'] /= n_fold
        return oof, prediction, np.mean(scores), np.std(scores), feature_importance
    else:
        return oof, prediction, np.mean(scores), np.std(scores)


# Feature selection

In [ ]:
n_fold_features = 5
folds_features = KFold(n_splits=n_fold_features, shuffle=True, random_state=11)

In [ ]:
params = {
    'num_leaves': 54,
    'min_data_in_leaf': 79,
    'objective': 'huber',
    'max_depth': -1,
    'learning_rate': 0.01,
    'boosting': 'gbdt',
    'bagging_freq': 5,
    'bagging_fraction': 0.8126672064208567,
    'bagging_seed': 11,
    'metric': 'mae',
    'verbosity': -1,
    'reg_alpha': 0.1302650970728192,
    'reg_lambda': 0.3603427518866501
}
oof_lgb, prediction_lgb, score_mean_lgb, score_std_lgb, feature_importance_lgb = train_model(
    X=X_train_scaled,
    X_test=X_test_scaled,
    y=y_tr,
    folds=folds_features,
    params=params,
    model_type='lgb',
    show_scatter=True)

After building a simple mode, let's see which are the important features

In [ ]:
max_num_features = 216
cols_feat = (feature_importance_lgb[['feature', 'importance']]
        .groupby('feature')
        .mean()
        .sort_values(by='importance', ascending=False)[:max_num_features].index)
best_features = feature_importance_lgb.loc[feature_importance_lgb.feature.isin(cols_feat)]

In [ ]:
plt.figure(figsize=(14,26))
sns.barplot(x='importance', y='feature', data=feature_importance_lgb.sort_values(by='importance', ascending=False))
plt.title('LightGBM Features (averaged over folds)')

TODO: Clearly, come features do not seem to bring any value to the mix. Let's visualize it carefully in case there is something wrong with them (or even if they make sense).

In [ ]:
best_features_list = cols_feat.tolist()
X_train_scaled_best = X_train_scaled[best_features_list]
X_test_scaled_best = X_test_scaled[best_features_list]
n_fold_models = 5
folds_models = KFold(n_splits=n_fold_models, shuffle=True, random_state=11)

## Models
Let's try a few different models and submit the one with the best validation score. The predicted values in the following plots are using a out-of-fold scheme.

### LGBM (Gradient Boosting)
Gradient boosting that uses tree based learning algorithms.

Let's first search for the best parameters for this model. It is not possible to use `GridSearchCV` with early stopping (lightgbm), so I am using a custom function for random search.

In [ ]:
fixed_params = {
    'objective': 'huber',
    'boosting': 'gbdt',
    'verbosity': -1,
    'random_seed': 19,
    'n_estimators': 50000,
    'metric': 'mae',
    'bagging_seed': 11
}

In [ ]:
param_grid = {
    'num_leaves': list(range(8, 92, 4)),
    'min_data_in_leaf': [10, 20, 40, 60, 100],
    'max_depth': [3, 4, 5, 6, 8, 12, 16, -1],
    'learning_rate': [0.1, 0.05, 0.01, 0.005],
    'bagging_freq': [3, 4, 5, 6, 7],
    'bagging_fraction': np.linspace(0.6, 0.95, 10),
    'reg_alpha': np.linspace(0.1, 0.95, 10),
    'reg_lambda': np.linspace(0.1, 0.95, 10)
}

In [ ]:
grid_size = 1
for param in param_grid:
    grid_size *= len(param_grid[param])
print(f'The search grid has {grid_size} elements')

In [ ]:
best_score = 9999
dataset = lgb.Dataset(X_train_scaled_best, label=y_tr)  # no need to scale features

scores_val_mean = []
scores_val_std = []
for i in tqdm_notebook(range(200)):
    params = {k: random.choice(v) for k, v in param_grid.items()}
    params.update(fixed_params)
    result = lgb.cv(params,
                    dataset,
                    nfold=n_fold_models,
                    early_stopping_rounds=200,
                    stratified=False)
    
    print(f"Iteration {i} finished with mae={result['l1-mean'][-1]:.4f} and std={result['l1-stdv'][-1]:.4f}")
    scores_val_mean.append(result['l1-mean'][-1])
    scores_val_std.append(result['l1-stdv'][-1])
    
    if result['l1-mean'][-1] < best_score:
        best_score = result['l1-mean'][-1]
        best_params = params

In [ ]:
plt.figure(figsize=(16, 6))
plt.scatter(scores_val_mean, scores_val_std, color='blue')
plt.scatter(best_score, best_score_std, color='gold')
plt.xlabel('scores_val_mean')
plt.ylabel('scores_val_std')
plt.title('Validation score mean/std scatter plot')
plt.grid()
plt.legend(['All parameters', 'Best'])
plt.show()

In [ ]:
print(f"best_score={best_score}")

In [ ]:
best_params

In [ ]:
params = {
    'objective': 'huber',
    'boosting': 'gbdt',
    'verbosity': -1,
    'num_leaves': 8, #54,
    'min_data_in_leaf': 100, #79,
    'max_depth': 6, #-1,
    'learning_rate': 0.01,
    'bagging_freq': 3, #5,
    'bagging_fraction': 0.7166666666666667, #0.8126672064208567,
    'bagging_seed': 11,
    'metric': 'mae',
    'reg_alpha': 0.19444444444444445, #0.1302650970728192,
    'reg_lambda': 0.8555555555555555 #0.3603427518866501
}
oof_lgb, prediction_lgb, score_mean_lgb, score_std_lgb, feature_importance = train_model(X=X_train_scaled_best,
                                                                                         X_test=X_test_scaled_best,
                                                                                         y=y_tr,
                                                                                         folds=folds_models,
                                                                                         params=params,
                                                                                         model_type='lgb',
                                                                                         show_scatter=True)

### XGBoost (Gradient Boosting)
Gradient boosting that uses tree based learning algorithms.

Let's first search for the best parameters for this model. It is not possible to use `GridSearchCV` with early stopping, so I am making my own gird search.

In [ ]:
fixed_params = {
    'objective': 'reg:linear',
    'eval_metric': 'mae',
    'silent': True,
    'nthread': 32
}

In [ ]:
param_grid = {
    'eta': [0.01, 0.015, 0.02, 0.025, 0.03, 0.1, 1, 5, 10], #Andrew uses 0.03
    'max_depth': [2, 4, 5, 6, 7, 8, 9, 10, 16, 32, 64, 128],
    'subsample': [0.1, 0.25, 0.5, 0.75, 0.9]
}

In [ ]:
grid_size = 1
for param in param_grid:
    grid_size *= len(param_grid[param])
print(f'The search grid has {grid_size} elements')

Random search on the parameter grid.

In [ ]:
best_score = 9999

dataset = xgb.DMatrix(data=X_train_scaled_best, label=y_tr, feature_names=X_train_scaled_best.columns)
scores_val_mean = []
scores_val_std = []
for i in tqdm_notebook(range(500)):
    params = {k: random.choice(v) for k, v in param_grid.items()}
    params.update(fixed_params)
    result = xgb.cv(params,
                    dataset,
                    nfold=n_fold_models,
                    num_boost_round=20000,
                    early_stopping_rounds=200,
                    stratified=False)
    
    print(f"Iteration {i} finished with mae={result['test-mae-mean'].iloc[-1]:.4f} and std={result['test-mae-std'].iloc[-1]:.4f}")
    scores_val_mean.append(result['test-mae-mean'].iloc[-1])
    scores_val_std.append(result['test-mae-std'].iloc[-1])
          
    if result['test-mae-mean'].iloc[-1] < best_score:
        best_score = result['test-mae-mean'].iloc[-1]
        best_score_std = result['test-mae-std'].iloc[-1]
        best_params = params        

In [ ]:
plt.figure(figsize=(16, 6))
plt.scatter(scores_val_mean, scores_val_std, color='blue')
plt.scatter(best_score, best_score_std, color='gold')
plt.xlim([2, 3])
plt.ylim([0, 0.1])
plt.xlabel('scores_val_mean')
plt.ylabel('scores_val_std')
plt.title('Validation score mean/std scatter plot')
plt.grid()
plt.legend(['All parameters', 'Best'])
plt.show()

In [ ]:
print(f"best_score={best_score}")

In [ ]:
best_params

In [ ]:
xgb_params = {
    'eta': 0.01, #Andrew uses 0.03
    'max_depth': 6, #Andrew uses 10
    'subsample': 0.5, #Andrew uses 0.9
    'objective': 'reg:linear',
    'eval_metric': 'mae',
    'silent': True,
    'nthread': 32
} # CV mean score: 2.0801, std: 0.0711.
oof_xgb, prediction_xgb, score_mean_xgb, score_std_xgb = train_model(X=X_train_scaled_best,
                                                                     X_test=X_test_scaled_best,
                                                                     y=y_tr,
                                                                     folds=folds_models,
                                                                     params=xgb_params,
                                                                     model_type='xgb',
                                                                     show_scatter=True)

### SVM
Similar to NuSVC, for regression, uses a parameter nu to control the number of support vectors. However, unlike NuSVC, where nu replaces C, here nu replaces the parameter epsilon of epsilon-SVR.

Let's first search for the best parameters ($\nu$ and $C$) for this model using `GridSearchCV`.

In [ ]:
grid = [{'nu': np.linspace(0.15, 0.99, 50),
         'C': np.linspace(0.15, 5, 50)}]

In [ ]:
target = y_tr.values.flatten()
params, cv_results = grid_search_cv(NuSVR(gamma='scale', tol=0.01), grid, X_train_scaled_best, target, n_fold_models)

In [ ]:
scores_mean = cv_results['mean_test_score'].reshape(len(grid[0]['nu']), len(grid[0]['C']))
scores_std = cv_results['std_test_score'].reshape(len(grid[0]['nu']), len(grid[0]['C']))

plt.subplots(figsize=(16, 6))
plt.subplot(1, 2, 1)
plt.imshow(scores_mean, interpolation='nearest', cmap=plt.cm.hot)
plt.xlabel('nu')
plt.ylabel('C')
plt.colorbar()
plt.xticks(np.arange(len(grid[0]['nu'])), grid[0]['nu'], rotation=45)
plt.yticks(np.arange(len(grid[0]['C'])), grid[0]['C'])
plt.title('Validation mean score')

plt.subplot(1, 2, 2)
plt.imshow(scores_std, interpolation='nearest', cmap=plt.cm.hot)
plt.xlabel('nu')
plt.ylabel('C')
plt.colorbar()
plt.xticks(np.arange(len(grid[0]['nu'])), grid[0]['nu'], rotation=45)
plt.yticks(np.arange(len(grid[0]['C'])), grid[0]['C'])
plt.title('Validation std score')
plt.show()

Now, we can put the best parameters in builidng the model.

In [ ]:
params

In [ ]:
#model = NuSVR(gamma='scale', nu=0.9, C=10.0, tol=0.01) #original values
model = NuSVR(gamma='scale', nu=0.63, C=0.4469387755102041, tol=0.01)
oof_svr, prediction_svr, score_mean_svr, score_std_svr = train_model(X=X_train_scaled_best,
                                                                     X_test=X_test_scaled_best,
                                                                     y=y_tr,
                                                                     folds=folds_models,
                                                                     params=None,
                                                                     model_type='sklearn',
                                                                     model=model,
                                                                     show_scatter=True)

### Catboost

In [ ]:
cat_params = {
    'loss_function':'MAE'
}
oof_cat, prediction_cat, score_mean_cat, score_std_cat = train_model(X=X_train_scaled_best,
                                                                     X_test=X_test_scaled_best,
                                                                     y=y_tr,
                                                                     folds=folds_models,
                                                                     params=cat_params,
                                                                     model_type='cat',
                                                                     show_scatter=True)

### Kernel Ridge
This model combines regularized linear regression with a given kernel (radial basis in this case).

Let's first search for the best parameters ($\alpha$ and $\gamma$) for this model using `GridSearchCV`.

In [ ]:
grid = [{
    'alpha': np.linspace(0.001, 10, 50),
    'gamma': np.linspace(0.00001, 0.03, 50)
}]

In [ ]:
target = y_tr.values.flatten()
params, cv_results = grid_search_cv(KernelRidge(kernel='rbf'), grid, X_train_scaled_best, target, n_fold_models)

In [ ]:
scores_mean = cv_results['mean_test_score'].reshape(len(grid[0]['alpha']), len(grid[0]['gamma']))
scores_std = cv_results['std_test_score'].reshape(len(grid[0]['alpha']), len(grid[0]['gamma']))

plt.subplots(figsize=(16, 6))
plt.subplot(1, 2, 1)
plt.imshow(scores_mean, interpolation='nearest', cmap=plt.cm.hot)
plt.xlabel('alpha')
plt.ylabel('gamma')
plt.colorbar()
plt.xticks(np.arange(len(grid[0]['alpha'])), grid[0]['alpha'], rotation=45)
plt.yticks(np.arange(len(grid[0]['gamma'])), grid[0]['gamma'])
plt.title('Validation accuracy')

plt.subplot(1, 2, 2)
plt.imshow(scores_std, interpolation='nearest', cmap=plt.cm.hot)
plt.xlabel('alpha')
plt.ylabel('gamma')
plt.colorbar()
plt.xticks(np.arange(len(grid[0]['alpha'])), grid[0]['alpha'], rotation=45)
plt.yticks(np.arange(len(grid[0]['gamma'])), grid[0]['gamma'])
plt.title('Validation accuracy')
plt.show()

In [ ]:
params

Now, we can put the best parameters in builidng the model.

In [ ]:
#model = KernelRidge(kernel='rbf', alpha=0.1, gamma=0.01) #Original parameters
model = KernelRidge(kernel='rbf', alpha=2.4497346938775513, gamma=0.0018461224489795917)
oof_r, prediction_r, score_mean_r, score_std_r = train_model(X=X_train_scaled_best,
                                                             X_test=X_test_scaled_best,
                                                             y=y_tr,
                                                             folds=folds_models,
                                                             params=None,
                                                             model_type='sklearn',
                                                             model=model,
                                                             show_scatter=True)

### Genetic Program Model
Genetic programming model from https://www.kaggle.com/scirpus/andrews-script-plus-a-genetic-program-model

In [ ]:
# There are no tuning here. We took at as face value.
oof_gdi, prediction_gdi, score_mean_gdi, score_std_gdi = train_model(X=X_train_scaled_best,
                                                                     X_test=X_test_scaled_best,
                                                                     y=y_tr,
                                                                     folds=folds_models,
                                                                     model_type='gdi',
                                                                     show_scatter=True)

### Random Forest
This regressor fits many decision trees with different subsets of the original data and average the predictions between them.

In [ ]:
grid = [{
    'max_depth': [8, 10, 12],
    'max_features': ['auto', 'sqrt', 'log2'],
    'min_samples_leaf': [2, 4, 8],
    'min_samples_split': [2, 4, 8]
}]

In [ ]:
target = y_tr.values.flatten()
params, cv_results = grid_search_cv(RandomForestRegressor(criterion='mae', n_estimators=200, n_jobs=-1),
                                    grid,
                                    X_train_scaled_best,
                                    target,
                                    n_fold_models)

In [ ]:
params

In [ ]:
rf_params = {
    'max_depth': 12, #8,
    'max_features': 'log2', #'auto',
    'min_samples_leaf': 2,
    'min_samples_split': 4 #6
} #CV mean score: 2.0448, std: 0.0793.
model = RandomForestRegressor(criterion='mae', n_estimators=200, n_jobs=-1, **rf_params)
oof_rf, prediction_rf, score_mean_rf, score_std_rf = train_model(X=X_train_scaled_best,
                                                                 X_test=X_test_scaled_best,
                                                                 y=y_tr,
                                                                 folds=folds_models,
                                                                 params=rf_params,
                                                                 model_type='sklearn',
                                                                 model=model,
                                                                 show_scatter=True)

### Extremely Randomized Trees


In [ ]:
grid = [{
    'max_depth': [5, 8, 10, 12],
    'max_features': ['auto', 'sqrt', 'log2'],
    'min_samples_leaf': [2, 4, 8],
    'min_samples_split': [2, 4, 8, 16]
}]

In [ ]:
target = y_tr.values.flatten()
params, cv_results = grid_search_cv(ExtraTreesRegressor(criterion='mae', n_estimators=200, n_jobs=-1),
                                    grid,
                                    X_train_scaled_best,
                                    target,
                                    n_fold_models)

In [ ]:
params

In [ ]:
ex_params = {
    'max_depth': 12,
    'max_features': 'sqrt',
    'min_samples_leaf': 2,
    'min_samples_split': 4
}
model = ExtraTreesRegressor(criterion='mae', n_estimators=200, n_jobs=-1, **ex_params)
oof_ex, prediction_ex, score_mean_ex, score_std_ex = train_model(X=X_train_scaled_best,
                                                                 X_test=X_test_scaled_best,
                                                                 y=y_tr,
                                                                 folds=folds_models,
                                                                 params=ex_params,
                                                                 model_type='sklearn',
                                                                 model=model,
                                                                 show_scatter=True)

### Ada Boost
AdaBoost begins by fitting a base estimator on the original dataset and then fits additional copies on the same dataset. At each iteration (estimator), the weights of instances are adjusted according to the error of the last prediction. It's similar to the next model, but gradient boosting fits additional estimator copies on the current error and not on the original dataset.

In [ ]:
grid = [{
    'learning_rate': np.linspace(0.0001, 0.1, 5)
}]
#base = Ridge(alpha=10)
base = KernelRidge(kernel='rbf', alpha=2.4497346938775513, gamma=0.0018461224489795917)
params = grid_search_cv(AdaBoostRegressor(base_estimator=base, n_estimators=100),
                        grid,
                        X_train_scaled,
                        target,
                        n_fold_models)


In [ ]:
params

In [ ]:
ada_params = {
    'learning_rate': 0.001
}
model = AdaBoostRegressor(base_estimator=base, n_estimators=100, **ada_params)
oof_ada, prediction_ada, score_mean_ada, score_std_ada = train_model(X=X_train_scaled_best,
                                                                     X_test=X_test_scaled_best,
                                                                     y=y_tr,
                                                                     folds=folds_models,
                                                                     params=ada_params,
                                                                     model_type='sklearn',
                                                                     model=model,
                                                                     show_scatter=True)

### NeuralNet
This is a simple FC model mimiquing the previous genetic algorithm.

In [ ]:
# Display training progress by printing a single dot for each completed epoch
class PrintDot(callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        if epoch % 100 == 0:
            print('')
        else:
            print('.', end='')

In [ ]:
def plot_history(history):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
  
    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('Mean Abs Error')
    plt.plot(hist['epoch'], hist['mean_absolute_error'],
             label='Train Error')
    plt.plot(hist['epoch'], hist['val_mean_absolute_error'],
             label = 'Val Error')
    plt.ylim([0,5])
    plt.legend()
    plt.grid()
    plt.show()

In [ ]:
param_grid = {
    'dropout': [0.4, 0.5, 0.7, 0.8, 0.9],
    'num_layers' : [2, 4, 6],
    'num_neurons' : [64, 128, 256]
}
grid = ParameterGrid(param_grid)
results_nn = pd.DataFrame(columns=['dropout', 'num_layers', 'num_neurons', 'val_score'])

for params in tqdm_notebook(grid):
    dropout = params['dropout']
    num_layers = params['num_layers']
    num_neurons = params['num_neurons']
    print(f"Training model with {params}")
    model = Sequential()
    model.add(layers.Dense(1024, input_dim=216, activation=tf.nn.relu))
    model.add(layers.Dropout(dropout))
    for l in range(num_layers):
        model.add(layers.Dense(num_neurons, input_dim=216, activation=tf.nn.relu))
        model.add(layers.Dropout(dropout))
    model.add(layers.Dense(1))

    model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])

    EPOCHS = 1000
    early_stop = callbacks.EarlyStopping(monitor='mean_absolute_error', patience=100)
    history = model.fit(
        X_train_scaled_best,
        y_tr,
        epochs=EPOCHS,
        validation_split=0.5,
        verbose=0,
        callbacks=[early_stop, PrintDot()])
    hist = pd.DataFrame(history.history)
    val_score = hist['val_mean_absolute_error'].iloc[-1]
    print(f'val_score={val_score}')
    plot_history(history)

    results_nn = results_nn.append({'dropout': dropout,
                                    'num_layers' : num_layers,
                                    'num_neurons' : num_neurons,
                                    'val_score' : val_score},
                                   ignore_index=True)
    

In [ ]:
results_nn

In [ ]:
plt.figure()
plt.scatter(results_nn.index, results_nn['val_score'])
plt.grid()

In [ ]:
del model

In [ ]:
sess = backend.get_session()
sess.close()
backend.clear_session()
gc.collect()
tf.keras.backend.clear_session()
sess = tf.keras.backend.get_session()
sess.close()
gc.collect()

In [ ]:
dropout = 0.4
num_layers = 3
num_neurons = 64
model = Sequential()
model.add(layers.Dense(1024, input_dim=216, activation=tf.nn.relu))
model.add(layers.Dropout(dropout))
for l in range(num_layers):
    model.add(layers.Dense(num_neurons, input_dim=216, activation=tf.nn.relu))
    model.add(layers.Dropout(dropout))
model.add(layers.Dense(1))

model = multi_gpu_model(model, gpus=2, cpu_merge=False)
model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
EPOCHS = 1000
early_stop = callbacks.EarlyStopping(monitor='mean_absolute_error', patience=100)
history = model.fit(
    X_train_scaled_best,
    y_tr,
    validation_split=0.5,
    epochs=EPOCHS,
    verbose=0,
    callbacks=[early_stop, PrintDot()])
hist = pd.DataFrame(history.history)
val_score = hist['val_mean_absolute_error'].iloc[-1]
print(f'val_score={val_score}')
plot_history(history)

results_nn = results_nn.append({'dropout': dropout,
                                'num_layers' : num_layers,
                                'num_neurons' : num_neurons,
                                'val_score' : val_score},
                               ignore_index=True)

In [ ]:
fig, axis = plt.subplots(1, 2, figsize=(12,5))
ax1, ax2 = axis
ax1.set_xlabel('actual')
ax1.set_ylabel('predicted')
ax2.set_xlabel('train index')
ax2.set_ylabel('time to failure')
        
ax1.scatter(y_tr, oof_nn, color='brown')
ax1.plot([(0, 0), (20, 20)], [(0, 0), (20, 20)], color='blue')

ax2.plot(y_tr, color='blue', label='y_train')
ax2.plot(oof_nn, color='orange')


In [ ]:
oof_nn, prediction_nn, score_mean_nn, score_std_nn = train_model(X=X_train_scaled_best,
                                                                 X_test=X_test_scaled_best,
                                                                 y=y_tr,
                                                                 folds=folds_models,
                                                                 params=params,
                                                                 model_type='nn',
                                                                 show_scatter=True)

## Stacking and blending
And now let's try stacking :) We can use the same function for it.

In [ ]:
train_stack = np.vstack([oof_lgb, oof_xgb, oof_svr, oof_r, oof_cat, oof_gdi, oof_rf, oof_ex,
                         oof_ada]).transpose()
train_stack = pd.DataFrame(train_stack, columns = ['lgb', 'xgb', 'svr', 'r', 'cat', 'gdi', 'rf', 'ex', 'ada'])
test_stack = np.vstack([prediction_lgb, prediction_xgb, prediction_svr, prediction_r,
                        prediction_cat, prediction_gdi, prediction_rf, prediction_ex, prediction_ada]).transpose()
test_stack = pd.DataFrame(test_stack)

In [ ]:
fixed_params = {
    'objective': 'huber',
    'boosting': 'gbdt',
    'verbosity': -1,
    'random_seed': 19,
    'n_estimators': 50000,
    'metric': 'mae',
    'bagging_seed': 11
}

In [ ]:
param_grid = {
    'num_leaves': list(range(8, 92, 4)),
    'min_data_in_leaf': [10, 20, 40, 60, 100],
    'max_depth': [3, 4, 5, 6, 8, 12, 16, -1],
    'learning_rate': [0.1, 0.05, 0.01, 0.005],
    'bagging_freq': [3, 4, 5, 6, 7],
    'bagging_fraction': np.linspace(0.6, 0.95, 10),
    'reg_alpha': np.linspace(0.1, 0.95, 10),
    'reg_lambda': np.linspace(0.1, 0.95, 10)
}

In [ ]:
best_score = 9999
dataset = lgb.Dataset(train_stack, label=y_tr)  # no need to scale features

scores_val_mean = []
scores_val_std = []
for i in tqdm_notebook(range(200)):
    params = {k: random.choice(v) for k, v in param_grid.items()}
    params.update(fixed_params)
    result = lgb.cv(params,
                    dataset,
                    nfold=n_fold_models,
                    early_stopping_rounds=200,
                    stratified=False)
    
    print(f"Iteration {i} finished with mae={result['l1-mean'][-1]:.4f} and std={result['l1-stdv'][-1]:.4f}")
    scores_val_mean.append(result['l1-mean'][-1])
    scores_val_std.append(result['l1-stdv'][-1])
    
    if result['l1-mean'][-1] < best_score:
        best_score = result['l1-mean'][-1]
        best_params = params

In [ ]:
plt.figure(figsize=(16, 6))
plt.scatter(scores_val_mean, scores_val_std, color='blue')
plt.scatter(best_score, best_score_std, color='gold')
plt.xlabel('scores_val_mean')
plt.ylabel('scores_val_std')
plt.title('Validation score mean/std scatter plot')
plt.grid()
plt.legend(['All parameters', 'Best'])
plt.show()

In [ ]:
print(f"best_score={best_score}")

In [ ]:
best_params

In [ ]:
params = {
    'objective': 'huber',
    'boosting': 'gbdt',
    'verbosity': -1,
    'num_leaves': 8, #54,
    'min_data_in_leaf': 100, #79,
    'max_depth': 5, #-1,
    'learning_rate': 0.01, #0.01,
    'bagging_freq': 5, #5,
    'bagging_fraction': 0.7555555555555555, #0.8126672064208567,
    'bagging_seed': 11,
    'metric': 'mae',
    'reg_alpha': 0.95, #0.1302650970728192,
    'reg_lambda': 0.19444444444444445 #0.3603427518866501
}
oof_lgb_stack, prediction_lgb_stack, score_mean_stack, score_std_stack, feature_importance = train_model(
    X=train_stack,
    X_test=test_stack,
    y=y_tr,
    folds=folds_models,
    params=params,
    model_type='lgb')

In [ ]:
cols = feature_importance[['feature', 'importance']].groupby('feature').mean().sort_values(
    by='importance', ascending=False).index
best_features = feature_importance.loc[feature_importance.feature.isin(cols)]
plt.figure(figsize=(16, 8));
sns.barplot(x='importance', y='feature', data=best_features.sort_values(by='importance', ascending=False));
plt.title('LGB Features (avg over folds)');

In [ ]:
plt.figure(figsize=(18, 18))
plt.subplot(4, 3, 1)
plt.plot(y_tr, label='y_train')
plt.plot(oof_lgb, label='lgb')
plt.legend()
plt.title('lgb');

plt.subplot(4, 3, 2)
plt.plot(y_tr, label='y_train')
plt.plot(oof_xgb, label='xgb')
plt.legend()
plt.title('xgb');

plt.subplot(4, 3, 3)
plt.plot(y_tr, label='y_train')
plt.plot(oof_svr, label='svr')
plt.legend()
plt.title('svr');

plt.subplot(4, 3, 4)
plt.plot(y_tr, label='y_train')
plt.plot(oof_r, label='r')
plt.legend()
plt.title('r');

plt.subplot(4, 3, 5)
plt.plot(y_tr, label='y_train')
plt.plot(oof_cat, label='cat')
plt.legend()
plt.title('cat');

plt.subplot(4, 3, 6)
plt.plot(y_tr, label='y_train')
plt.plot(oof_gdi, label='gdi')
plt.legend()
plt.title('gdi');

plt.subplot(4, 3, 7)
plt.plot(y_tr, label='y_train')
plt.plot(oof_rf, label='rf')
plt.legend()
plt.title('rf');

plt.subplot(4, 3, 8)
plt.plot(y_tr, label='y_train')
plt.plot(oof_ex, label='ex')
plt.legend()
plt.title('ex');

plt.subplot(4, 3, 9)
plt.plot(y_tr, label='y_train')
plt.plot(oof_ada, label='ada')
plt.legend()
plt.title('ada');

plt.subplot(4, 3, 11)
plt.plot(y_tr, label='y_train')
plt.plot(oof_lgb_stack, label='stack')
plt.legend()
plt.title('stacked');

plt.subplot(4, 3, 12)
plt.plot(y_tr, label='y_train')
plt.plot((oof_lgb + oof_xgb + oof_svr + oof_r + oof_cat + oof_gdi + oof_rf + oof_ex + oof_ada) / 9, label='blend')
plt.legend()
plt.title('blend');
plt.suptitle('Predictions vs actual');

### Submission - Blend model

In [ ]:
submission['time_to_failure'] = (prediction_lgb + prediction_xgb + prediction_svr
                                + prediction_cat + prediction_r + prediction_gdi
                                + prediction_rf + prediction_ex + prediction_ada) / 9
submission.to_csv('../output/submission_rdg_notebook3_blend.csv')
score_mean_blend = (score_mean_lgb + score_mean_xgb + score_mean_svr + score_mean_cat
                    + score_mean_r + score_mean_gdi + score_mean_rf + score_mean_ex + score_mean_ada) / 9
score_std_blend = (score_std_lgb + score_std_xgb + score_std_svr + score_std_cat
                    + score_std_r + score_std_gdi + score_std_rf + score_std_ex + score_std_ada) / 9
print(f'CV mean score: {score_mean_blend:.4f}, std: {score_std_blend:.4f}.')

### Submission - Stacked model

In [ ]:
submission['time_to_failure'] = prediction_lgb_stack
submission.to_csv('../output/submission_rdg_notebook3_stack.csv')
print(f'CV mean score: {score_mean_stack:.4f}, std: {score_std_stack:.4f}.')

### Submission - LGB model

In [ ]:
submission['time_to_failure'] = prediction_lgb
submission.to_csv('../output/submission_rdg_notebook3_lgb.csv')
print(f'CV mean score: {score_mean_lgb:.4f}, std: {score_std_lgb:.4f}.')

### Submission - XGB model

In [ ]:
submission['time_to_failure'] = prediction_xgb
submission.to_csv('../output/submission_rdg_notebook3_xgb.csv')
print(f'CV mean score: {score_mean_xgb:.4f}, std: {score_std_xgb:.4f}.')

### Submission - SVR model

In [ ]:
submission['time_to_failure'] = prediction_svr
submission.to_csv('../output/submission_rdg_notebook3_svr.csv')
print(f'CV mean score: {score_mean_svr:.4f}, std: {score_std_svr:.4f}.')

### Submission - CAT model

In [ ]:
submission['time_to_failure'] = prediction_cat
submission.to_csv('../output/submission_rdg_notebook3_cat.csv')
print(f'CV mean score: {score_mean_cat:.4f}, std: {score_std_cat:.4f}.')

### Submission - R model

In [ ]:
submission['time_to_failure'] = prediction_r
submission.to_csv('../output/submission_rdg_notebook3_r.csv')
print(f'CV mean score: {score_mean_r:.4f}, std: {score_std_r:.4f}.')

### Submission - GDI model

In [ ]:
submission['time_to_failure'] = prediction_gdi
submission.to_csv('../output/submission_rdg_notebook3_gdi.csv')
print(f'CV mean score: {score_mean_gdi:.4f}, std: {score_std_gdi:.4f}.')

### Submission - RF model

In [ ]:
submission['time_to_failure'] = prediction_rf
submission.to_csv('../output/submission_rdg_notebook3_rf.csv')
print(f'CV mean score: {score_mean_rf:.4f}, std: {score_std_rf:.4f}.')

### Submission - EX model

In [ ]:
submission['time_to_failure'] = prediction_ex
submission.to_csv('../output/submission_rdg_notebook3_ex.csv')
print(f'CV mean score: {score_mean_ex:.4f}, std: {score_std_ex:.4f}.')

### Submission - ADA model

In [ ]:
submission['time_to_failure'] = prediction_ada
submission.to_csv('../output/submission_rdg_notebook3_ada.csv')
print(f'CV mean score: {score_mean_ada:.4f}, std: {score_std_ada:.4f}.')

## Models overview

In [ ]:
evaluation = pd.DataFrame(dtype=np.float64)

In [ ]:
evaluation.loc['lgb', 'mean'] = score_mean_lgb
evaluation.loc['lgb', 'std'] = score_std_lgb

evaluation.loc['xgb', 'mean'] = score_mean_xgb
evaluation.loc['xgb', 'std'] = score_std_xgb

evaluation.loc['svr', 'mean'] = score_mean_svr
evaluation.loc['svr', 'std'] = score_std_svr

evaluation.loc['cat', 'mean'] = score_mean_cat
evaluation.loc['cat', 'std'] = score_std_cat

evaluation.loc['r', 'mean'] = score_mean_r
evaluation.loc['r', 'std'] = score_std_r

evaluation.loc['gdi', 'mean'] = score_mean_gdi
evaluation.loc['gdi', 'std'] = score_std_gdi

evaluation.loc['rf', 'mean'] = score_mean_rf
evaluation.loc['rf', 'std'] = score_std_rf

evaluation.loc['ex', 'mean'] = score_mean_ex
evaluation.loc['ex', 'std'] = score_std_ex

evaluation.loc['ada', 'mean'] = score_mean_ada
evaluation.loc['ada', 'std'] = score_std_ada

evaluation.loc['blend', 'mean'] = score_mean_blend
evaluation.loc['blend', 'std'] = score_std_blend

evaluation.loc['stack', 'mean'] = score_mean_stack
evaluation.loc['stack', 'std'] = score_std_stack

In [ ]:
evaluation

In [ ]:

evaluation.to_csv(f"../output/evaluation_{str(datetime.datetime.now())}.csv")

In [ ]:
plt.subplots(figsize=(18, 10))
plt.subplot(2, 1, 1)
plt.bar(np.arange(len(evaluation.index.tolist())), evaluation['mean'], yerr=evaluation['std'], align='center',
       alpha=0.5, ecolor='black', capsize=10)
plt.ylabel('CV score')
plt.xlabel('model')
plt.xticks(np.arange(len(evaluation.index.tolist())), evaluation.index.tolist())
plt.grid()

plt.subplot(2, 1, 2)
plt.scatter(evaluation['mean'], evaluation['std'])

model_list = evaluation.index.tolist()
score_mean_list = evaluation['mean'].tolist()
score_std_list = evaluation['std'].tolist()

for i in range(len(model_list)):
    plt.annotate(model_list[i], (score_mean_list[i]+0.001, score_std_list[i]))
    
plt.ylabel('std')
plt.xlabel('mean')
plt.grid()

plt.show()